In [ ]:
import argparse
import logging
import os
import sys
import joblib

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from datetime import datetime

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234321)
skf.get_n_splits(

for fold, (train_index, test_index) in enumerate(skf.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [4]:
import pandas as pd
df = pd.read_parquet('raw_apoe_AD.parquet', engine = 'fastparquet')
df = df['IID'].to_frame() # just the id's, no snps
df.head()

,IID
0,3733928
1,3607009
2,2546462
3,4886993
4,2409738


In [5]:
lancets = pd.read_parquet('../../../randy/rfb/tidy_data/UKBiobank/dementia/lancet2024/lancet2024_preprocessed.parquet', engine = 'fastparquet')

In [10]:
lancet_vars = [
        "4700-0.0",
        "5901-0.0",
        "30780-0.0",
        "head_injury",
        "22038-0.0",
        "20161-0.0",
        "alcohol_consumption",
        "hypertension",
        "obesity",
        "diabetes",
        "hearing_loss",
        "depression",
        "freq_friends_family_visit",
        "24012-0.0",
        "24018-0.0",
        "24019-0.0",
        "24006-0.0",
        "24015-0.0",
        "24011-0.0",
        "2020-0.0_-3.0",
        "2020-0.0_-1.0",
        "2020-0.0_0.0",
        "2020-0.0_1.0",
        "2020-0.0_nan",
    ]

overlap = [col for col in lancet_vars if col in lancets.columns]
print("Overlap columns:", overlap)

Overlap columns: ['4700-0.0', '5901-0.0', '30780-0.0', 'head_injury', '22038-0.0', '20161-0.0', 'alcohol_consumption', 'hypertension', 'obesity', 'diabetes', 'hearing_loss', 'depression', 'freq_friends_family_visit', '24012-0.0', '24018-0.0', '24019-0.0', '24006-0.0', '24015-0.0', '24011-0.0']


In [ ]:
def settings_automl(time_budget, metric, model):
    """
    Generate settings for an AutoML classification task.
    Parameters:
    time_budget (int): The time budget for the AutoML process in seconds.
    metric (str): The evaluation metric to be used (e.g., 'log_loss' 'accuracy', 'f1').
    model (str): The model to be used in the AutoML process (e.g., 'lrl1').
    region_index (int): The index of the region for logging purposes.
    Returns:
    dict: A dictionary containing the settings for the AutoML process.
    """
    automl_settings = {
            "task": "classification",
            "time_budget": time_budget,
            "metric": 'log_loss',
            "n_jobs": -1,
            "eval_method": "cv",
            "n_splits": 5,
            "early_stop": True,
            "log_training_metric": True,
            "model_history": True,
            "seed": 1234321,
            "estimator_list": [model],
    }

    return automl_settings

In [ ]:
def main(): 
    FLAML_TIME_BUDGET = 3600
    
    #print(os.getcwd())
    snps = pd.read_parquet("raw_snps_AD.parquet") #what is IDP??

    # set up logging
    logging.basicConfig(filename=f"{results_dir}/logging.txt", level=logging.INFO)
    logging.info(
        "Running experiment with snps",
        snps
    )
    logging.info("X shape: %s", X.shape)
    logging.info("y shape: %s", y.shape)

    